# 存储

## 概念

LlamaIndex 提供了一个用于提取、索引和查询外部数据的高级接口。

在底层，LlamaIndex 还支持可交换的存储组件，允许您自定义：

- 文档存储Node：存储摄取的文档（即对象）的位置，
- 索引存储：存储索引元数据的地方，
- 向量存储：存储嵌入向量的地方
- 图形存储：存储知识图谱的地方（即KnowledgeGraphIndex）。
- 聊天存储：存储和组织聊天信息的地方。

文档/索引存储依赖于通用的键值存储抽象.

LlamaIndex 支持将数据持久保存到fsspec支持的任何存储后端。我们已确认支持以下存储后端：

- 本地文件系统
- AWS S3
- Cloudflare R2

![](https://docs.llamaindex.ai/en/stable/_static/storage/storage.png)

## 使用模式

许多向量存储（FAISS 除外）将同时存储数据和索引（嵌入）。

这意味着您不需要使用单独的文档存储或索引存储。

这也意味着您不需要明确保留这些数据 - 这是自动发生的。

使用方法类似于以下内容来构建新索引/重新加载现有索引。

In [ ]:
## build a new index
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.deeplake import DeepLakeVectorStore

# construct vector store and customize storage context
vector_store = DeepLakeVectorStore(dataset_path="<dataset_path>")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
# Load documents and build index
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)


## reload an existing one
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

通常要使用存储抽象，您需要定义一个StorageContext对象：

In [ ]:
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.storage.index_store import SimpleIndexStore
from llama_index.core.vector_stores import SimpleVectorStore
from llama_index.core import StorageContext

# create storage context using default stores
storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore(),
    vector_store=SimpleVectorStore(),
    index_store=SimpleIndexStore(),
)

## 向量存储

向量存储包含摄取的文档块的嵌入向量（有时也包含文档块）。

### 简单向量存储 - Simple Vector Store

默认情况下，LlamaIndex 使用简单的内存向量存储，非常适合快速实验。

可以通过调用 vector_store.persist() （和分别调用 SimpleVectorStore.from_persist_path(...) ）将它们持久保存到磁盘（并从磁盘加载）。

### 向量存储选项和功能支持

LlamaIndex 支持 20 多种不同的矢量存储选项。我们正在积极添加更多集成并改进每种选项的功能覆盖范围。

（表略）

## 文档存储

文档存储包含摄取的文档块，我们称之为Node对象。

### 简单文档存储 - SimpleDocumentStore

默认情况下，将对象SimpleDocumentStore存储Node在内存中。

通过分别调用docstore.persist()（和SimpleDocumentStore.from_persist_path(...)）将它们保存到磁盘（并从磁盘加载）。

### MongoDB 文档存储 - MongoDocumentStore

我们支持 MongoDB 作为替代文档存储后端，其在Node对象被摄取时保存数据。

In [ ]:
from llama_index.storage.docstore.mongodb import MongoDocumentStore
from llama_index.core.node_parser import SentenceSplitter

# create parser and parse document into nodes
parser = SentenceSplitter()
nodes = parser.get_nodes_from_documents(documents)

# create (or load) docstore and add nodes
docstore = MongoDocumentStore.from_uri(uri="<mongodb+srv://...>")
docstore.add_documents(nodes)

# create storage context
storage_context = StorageContext.from_defaults(docstore=docstore)

# build index
index = VectorStoreIndex(nodes, storage_context=storage_context)

在底层，MongoDocumentStore连接到固定的 MongoDB 数据库并为您的节点初始化新集合（或加载现有集合）。

使用 MongoDocumentStore 时不需要调用storage_context.persist()，因为数据默认是持久的。

可以轻松地重新连接到您的 MongoDB 集合，并通过使用现有的db_name和重新初始化来重新加载索引collection_name。

### Redis 文档存储

支持 Redis 作为替代文档存储后端，其在Node提取对象时持久保存数据。

In [ ]:
from llama_index.storage.docstore.redis import RedisDocumentStore
from llama_index.core.node_parser import SentenceSplitter

# create parser and parse document into nodes
parser = SentenceSplitter()
nodes = parser.get_nodes_from_documents(documents)

# create (or load) docstore and add nodes
docstore = RedisDocumentStore.from_host_and_port(
    host="127.0.0.1", port="6379", namespace="llama_index"
)
docstore.add_documents(nodes)

# create storage context
storage_context = StorageContext.from_defaults(docstore=docstore)

# build index
index = VectorStoreIndex(nodes, storage_context=storage_context)

在底层，RedisDocumentStore连接到 redis 数据库并将您的节点添加到存储在下的命名空间{namespace}/docs。

RedisDocumentStore您可以轻松地重新连接到 Redis 客户端，并通过使用现有的host、port和重新初始化 来重新加载索引namespace。

### Firestore 文档存储

支持 Firestore 作为替代文档存储后端，它在Node提取对象时保留数据。

FirestoreDocumentStore连接到 Google Cloud 中的 firestore 数据库，并将您的节点添加到存储在下的命名空间中{namespace}/docs。

## 索引存储

索引存储包含轻量级索引元数据（即构建索引时创建的附加状态信息）。

### 简单索引存储 - SimpleIndexStore

默认情况下，LlamaIndex 使用由内存中的键值存储支持的简单索引存储。

可以通过分别调用（和）保存到磁盘（并从磁盘加载index_store.persist()）SimpleIndexStore.from_persist_path(...)。

### MongoDB 索引存储

与文档存储类似，我们也可以将其用作MongoDB索引存储的存储后端。

### Redis 索引存储 - RedisIndexStore

支持 Redis 作为替代文档存储后端，其在Node提取对象时持久保存数据。

## 对话存储

聊天存储是存储聊天记录的集中界面。

与其他存储格式相比，聊天记录是独一无二的，因为消息的顺序对于维护整体对话非常重要。

user_ids聊天存储器可以通过键（如或其他唯一可识别的字符串）组织聊天消息序列，并处理delete、insert和get操作

### 简单对话存储

最基本的聊天存储是SimpleChatStore，它将消息存储在内存中并可以保存到磁盘/从磁盘保存，或者可以序列化并存储在其他地方。

SimpleChatStore 通常，您将实例化对话存储并将其提供给内存模块。如果未提供聊天存储，则使用聊天存储的内存模块将默认使用。

In [ ]:
from llama_index.core.storage.chat_store import SimpleChatStore
from llama_index.core.memory import ChatMemoryBuffer

chat_store = SimpleChatStore()

chat_memory = ChatMemoryBuffer.from_defaults(
    token_limit=3000,
    chat_store=chat_store,
    chat_store_key="user1",
)

创建了记忆，您就可以将其包含在代理或聊天引擎中：

In [ ]:
agent = OpenAIAgent.from_tools(tools, memory=memory)
# OR
chat_engine = index.as_chat_engine(memory=memory)

要保存聊天记录以供日后使用，您可以从磁盘保存或加载

In [ ]:
chat_store.persist(persist_path="chat_store.json")
loaded_chat_store = SimpleChatStore.from_persist_path(
    persist_path="chat_store.json"
)

或者你可以转换为字符串，并将字符串保存在其他地方

In [ ]:
chat_store_string = chat_store.json()
loaded_chat_store = SimpleChatStore.parse_raw(chat_store_string)

### RedisChatStore

使用RedisChatStore，您可以远程存储您的聊天记录，而不必担心手动保存和加载聊天记录。

### AzureChatStore

使用AzureChatStore，您可以将聊天记录远程存储在 Azure 表存储或 CosmosDB 中，而不必担心手动保存和加载聊天记录。

### 键值存储

键值存储是支持我们的文档存储和索引存储的底层存储抽象。

提供以下键值存储：

- Simple Key-Value Store: 内存中的 KV 存储。用户可以选择调用persist此 KV 存储将数据持久保存到磁盘。
- MongoDB Key-Value Store: MongoDB KV 存储。

注意：目前，这些存储抽象并不面向外部。

## 保存并加载数据

### 持久数据

默认情况下，LlamaIndex 将数据存储在内存中，如果需要，可以明确地保留这些数据：

In [ ]:
storage_context.persist(persist_dir="<persist_dir>")

按照指定的方式persist_dir（或./storage默认方式）将数据保存到磁盘。

### 加载数据

要加载数据，用户只需使用相同的配置（例如传入相同persist_dir或向量存储客户端）重新创建存储上下文。

In [ ]:
storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="<persist_dir>"),
    vector_store=SimpleVectorStore.from_persist_dir(
        persist_dir="<persist_dir>"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="<persist_dir>"),
)

可以通过下面的一些便利函数加载特定索引

In [ ]:
from llama_index.core import (
    load_index_from_storage,
    load_indices_from_storage,
    load_graph_from_storage,
)

# load a single index
# need to specify index_id if multiple indexes are persisted to the same directory
index = load_index_from_storage(storage_context, index_id="<index_id>")

# don't need to specify index_id if there's only one index in storage context
index = load_index_from_storage(storage_context)

# load multiple indices
indices = load_indices_from_storage(storage_context)  # loads all indices
indices = load_indices_from_storage(
    storage_context, index_ids=[index_id1, ...]
)  # loads specific indices

# load composable graph
graph = load_graph_from_storage(
    storage_context, root_id="<root_id>"
)  # loads graph with the specified root_id

### 使用远程后端

默认情况下，LlamaIndex 使用本地文件系统来加载和保存文件。

可以通过传递fsspec.AbstractFileSystem对象来覆盖此设置。

实例化一个 S3 文件系统并从那里保存/加载：

In [ ]:
# set up s3fs
AWS_KEY = os.environ["AWS_ACCESS_KEY_ID"]
AWS_SECRET = os.environ["AWS_SECRET_ACCESS_KEY"]
R2_ACCOUNT_ID = os.environ["R2_ACCOUNT_ID"]

assert AWS_KEY is not None and AWS_KEY != ""

s3 = s3fs.S3FileSystem(
    key=AWS_KEY,
    secret=AWS_SECRET,
    endpoint_url=f"https://{R2_ACCOUNT_ID}.r2.cloudflarestorage.com",
    s3_additional_kwargs={"ACL": "public-read"},
)

# If you're using 2+ indexes with the same StorageContext,
# run this to save the index to remote blob storage
index.set_index_id("vector_index")

# persist index to s3
s3_bucket_name = "llama-index/storage_demo"  # {bucket_name}/{index_name}
index.storage_context.persist(persist_dir=s3_bucket_name, fs=s3)

# load index from s3
index_from_s3 = load_index_from_storage(
    StorageContext.from_defaults(persist_dir=s3_bucket_name, fs=s3),
    index_id="vector_index",
)

## 自定义存储

默认情况下，LlamaIndex 隐藏了复杂性，让您用不到 5 行代码查询数据：

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()
response = query_engine.query("Summarize the documents.")

在底层，LlamaIndex 还支持可交换的存储层，允许您自定义摄取的文档（即Node对象）、嵌入向量和索引元数据的存储位置。

![](https://docs.llamaindex.ai/en/stable/_static/storage/storage.png)

### 底层 API

In [ ]:
# 高级 API
index = VectorStoreIndex.from_documents(documents)

使用可以提供更精细控制的低级 API：

### 向量存储的集成和存储

In [ ]:
大多数向量存储集成将整个索引（矢量 + 文本）存储在矢量存储本身中。

向量存储已经托管并持久化了索引中的数据。